In [ ]:
import pandas as pd
from collections import defaultdict
from itertools import combinations
import ast

c:\Users\liang\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\liang\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Preprocess Data

In [2]:
import pandas as pd
import re

def preprocess_triplegs(df, timestamp_col='started_at'):
    """
    Preprocess the triplegs data by extracting sequences of triplegs for each user.

    Parameters:
    df (pd.DataFrame): The triplegs DataFrame.
    timestamp_col (str): The name of the column containing the timestamp.

    Returns:
    list: A list of sequences of triplegs for each user as lists of tuples.
    """
    # Convert the timestamp column to datetime
    df[timestamp_col] = pd.to_datetime(df[timestamp_col])

    # Define the starting date and filter data to include only the first 30 days
    start_date = pd.to_datetime('1900-01-01').tz_localize('UTC')
    end_date = start_date + pd.Timedelta(days=30)

    # Use .copy() to avoid SettingWithCopyWarning
    df = df.loc[(df[timestamp_col] >= start_date) & (df[timestamp_col] < end_date)].copy()

    # Function to convert LINESTRING to integer coordinates
    def linestring_to_int_coords(linestring):
        # Extract the coordinates from the LINESTRING
        coords = re.findall(r'(\d+\.\d+ \d+\.\d+)', linestring)
        # Convert the coordinates to tuples of integers
        int_coords = [tuple(map(int, map(float, coord.split()))) for coord in coords]
        return int_coords  # Return as a list of tuples

    # Convert the LINESTRING sequences to integer coordinates
    df['geom'] = df['geom'].apply(linestring_to_int_coords)

    # Extract sequences of triplegs for each user
    user_sequences = df.groupby('user_id')['geom'].apply(list).tolist()

    # Flatten the list of lists into a single list of tuples
    final_output = []
    for sequence in user_sequences:
        final_output.extend(sequence)  

    return final_output

In [3]:
def split_long_triplegs(df, max_length=1000):
    """
    Split long triplegs into shorter sub-triplegs.

    Parameters:
    df (pd.DataFrame): The triplegs DataFrame.
    max_length (int): The maximum length of a tripleg.

    Returns:
    pd.DataFrame: The DataFrame with long triplegs split into shorter sub-triplegs.
    """
    def split_geom(geom, max_length):
        points = geom.split(',')
        sub_triplegs = [','.join(points[i:i + max_length]) for i in range(0, len(points), max_length)]
        return sub_triplegs

    new_rows = []
    for _, row in df.iterrows():
        geoms = split_geom(row['geom'], max_length)
        for geom in geoms:
            new_row = row.copy()
            new_row['geom'] = geom
            new_rows.append(new_row)

    return pd.DataFrame(new_rows)

## Apply GSP

In [4]:
from collections import defaultdict
from itertools import combinations

#contiguous matching
def generate_candidates(sequences, length):
    """Generate candidate sequences of a given length."""
    candidates = set()
    for seq in sequences:
        for i in range(len(seq) - length + 1):
            candidates.add(tuple(seq[i:i + length]))
    return candidates

def is_subsequence(candidate, sequence):
    """Check if candidate is a subsequence of sequence."""
    it = iter(sequence)
    return all(item in it for item in candidate)

#discontinuous matching
# def generate_candidates(sequences, length):
#     """Generate candidate sequences of a given length, allowing non-contiguous elements."""
#     candidates = set()
#     for seq in sequences:
#         for indices in combinations(range(len(seq)), length):
#             # Create a candidate using the indices to select elements from the sequence
#             candidates.add(tuple(tuple(seq[i]) for i in indices))
#     return candidates

# def is_subsequence(candidate, sequence):
#     """Check if the candidate sequence is a subsequence of the sequence, ignoring elements in between."""
#     it = iter(sequence)  # Create an iterator for the sequence
#     return all(item in it for item in candidate)

def count_support(candidates, sequences):
    """Count the support of each candidate sequence in the dataset."""
    support_count = defaultdict(int)
    for candidate in candidates:
        for seq in sequences:
            if is_subsequence(candidate, seq):
                support_count[candidate] += 1
    return support_count


def prune_candidates(support_count, min_support):
    """Prune candidate sequences that do not meet the minimum support threshold."""
    return {seq: count for seq, count in support_count.items() if count >= min_support}

def generate_new_candidates(frequent_sequences, length):
    """Generate new candidate sequences by joining frequent sequences."""
    new_candidates = set()
    frequent_sequences = list(frequent_sequences)
    for seq1, seq2 in combinations(frequent_sequences, 2):
        # Join sequences if they can form a new length candidate
        if seq1[1:] == seq2[:-1]:
            new_candidates.add(seq1 + (seq2[-1],))
    return new_candidates

def gsp(sequences, min_support):
    """Implement the GSP algorithm to mine sequential patterns."""
    length = 2
    frequent_sequences = generate_candidates(sequences, length)
    all_frequent_sequences = []

    while frequent_sequences:
        support_count = count_support(frequent_sequences, sequences)
        frequent_sequences = prune_candidates(support_count, min_support)

        all_frequent_sequences.extend(frequent_sequences.keys())

        length += 1
        frequent_sequences = generate_new_candidates(frequent_sequences.keys(), length)

    return all_frequent_sequences


## Save Output to CSV

In [ ]:
def save_gsp_results(gsp_results, output_file):
    """
    Save the GSP results to a CSV file.

    Parameters:
    gsp_results (list): A list of frequent sequences.
    output_file (str): The path to the output CSV file.
    """
    sequences_df = pd.DataFrame(gsp_results, columns=['Sequence'])
    sequences_df.to_csv(output_file, index=False)
    print(f"GSP results saved to {output_file}")

In [ ]:
df = pd.read_csv('./triplegsA.csv' , compression='zip')

# Split long triplegs into shorter sub-triplegs
df = split_long_triplegs(df)

# Preprocess the data and limit to the first month
sequences = preprocess_triplegs(df)

In [ ]:
# Define the minimum support threshold dynamically as x% of the dataset
min_support = int(0.00001 * len(sequences)) 
print(f"Minimum support threshold: {min_support}")

# Apply the GSP algorithm to find frequent sequences
frequent_sequences = gsp(sequences, min_support)

save_gsp_results(frequent_sequences, 'frequent_sequences.csv')

## Additional Cell to calculate the average sequence length

In [ ]:

# Read the CSV file
df = pd.read_csv('frequent_sequences_A.csv')

# Function to calculate the length of a pattern
def calculate_pattern_length(pattern):
    # Parse the pattern string into a list of tuples
    pattern_list = ast.literal_eval(pattern)
    # Return the length of the pattern
    return len(pattern_list)

# Apply the function to calculate the length of each pattern
df['Pattern_Length'] = df['Pattern'].apply(calculate_pattern_length)

# Calculate the average length of the patterns
average_length = df['Pattern_Length'].mean()

print(f'The average length of the patterns is: {average_length}')